In [1]:
import numpy as np

In [106]:
mat = [[-3, 6, -1, 1, -7], [1, -2, 2, 3, -1], [2, -4, 5, 8, -4]]
mat = np.array(mat, dtype=np.double)

In [139]:
mat

array([[-3.,  6., -1.,  1., -7.],
       [ 1., -2.,  2.,  3., -1.],
       [ 2., -4.,  5.,  8., -4.]])

In [132]:
m, n = mat.shape

In [136]:
def count_zeros(arr):
    zeros = []
    arr = np.round(arr, 5)
    for row in arr:
        for ind, element in enumerate(row):
            if element != 0:
                zeros.append(ind)
                break
            elif ind == arr.shape[1] - 1:
                zeros.append(ind + 1)
                break

    return zeros

def sort_by_zeros(arr):
    zeros = count_zeros(arr)
    arr_sorted = arr[np.argsort(zeros)]

    return np.sort(zeros), arr_sorted

def rr_down(sliced):
    m, n = sliced.shape
    row_zero = (sliced[0, :] / sliced[0, 0])[..., np.newaxis].T
    coefs_mat = np.repeat(a = row_zero, repeats = m - 1, axis = 0) * (-1 * sliced[1:, 0][:, np.newaxis])
    sliced[1:, :] += coefs_mat

def rr_up(sliced):
    m, n = sliced.shape
    sliced[-1, :] /= sliced[-1, 0]
    row_last = sliced[-1, :][..., np.newaxis].T
    coefs_mat = np.repeat(a = row_last, repeats = m - 1, axis = 0) * (-1 * sliced[:-1, 0][:, np.newaxis])
    sliced[:-1, :] += coefs_mat

def echelon_form(mat):
    mat = mat.copy()
    for i in range(mat.shape[0] - 1):
        sliced = mat[i:, :]
        zeros, sliced = sort_by_zeros(sliced)

        pivot_position = zeros[0]
        rr_down(sliced[:, pivot_position:])
        mat[i:, :] = sliced
    return mat

def reduced_echelon_form(mat):
    ef = mat.copy()
    zeros = count_zeros(ef)
    
    m, n = ef.shape
    for i in reversed(range(m)):
        sliced = ef[:i + 1, :]
        
        pivot_position = zeros[i]
        if pivot_position == n:
            continue
        
        rr_up(sliced[:, pivot_position:])
        ef[:i + 1, :] = sliced
    return ef

In [140]:
ef = echelon_form(mat)
print(np.round(ef, 3))
zeros = count_zeros(ef)
zeros

[[-3.     6.    -1.     1.    -7.   ]
 [ 0.     0.     1.667  3.333 -3.333]
 [ 0.     0.     0.     0.    -0.   ]]


[0, 2, 5]

In [179]:
ref = reduced_echelon_form(ef)
print(np.round(ref, 3))

[[ 1. -2. -0. -1.  3.]
 [ 0.  0.  1.  2. -2.]
 [ 0.  0.  0.  0. -0.]]


In [145]:
pivots = np.array(zeros)
pivots = pivots[pivots != n]

In [147]:
pivots

array([0, 2])

In [135]:
ef[pivots != n - 1]

array([[-3.        ,  6.        , -1.        ,  1.        , -7.        ],
       [ 0.        ,  0.        ,  1.66666667,  3.33333333, -3.33333333]])

In [189]:
def get_null_space(ref, pivots):
    m, n = ref.shape
    null_space = np.zeros((n, n - pivots.shape[0]))
    for ref_row_index in range(pivots.shape[0]):
        pivot = pivots[ref_row_index]
        ref_row = ref[ref_row_index, pivot+1:]
        for i, e in enumerate(ref_row[ref_row != 0]):
            null_space[pivot, ref_row_index + i] = -1 * e
            
    col = 0
    for row in [i for i in range(n) if i not in pivots.tolist()]:
        null_space[row, col] = 1
        col += 1
    return null_space

def get_row_space(ref, pivots):
    return ref[:pivots.shape[0], :].T

def get_col_space(mat, pivots):
    return mat[:, pivots]

In [184]:
get_null_space(ref, pivots)

array([[ 2.,  1., -3.],
       [ 1.,  0.,  0.],
       [ 0., -2.,  2.],
       [ 0.,  1.,  0.],
       [ 0.,  0.,  1.]])

In [186]:
get_row_space(ref, pivots)

array([[ 1.,  0.],
       [-2.,  0.],
       [-0.,  1.],
       [-1.,  2.],
       [ 3., -2.]])

In [190]:
get_col_space(mat, pivots)

array([[-3., -1.],
       [ 1.,  2.],
       [ 2.,  5.]])